In [1]:
from __future__ import print_function
import torch.distributed as dist 
import torch.nn as nn
import LIAF,LIAFnet
import argparse, pickle, torch, time, os,sys
from importlib import import_module
from LIAFnet.LIAFResNet import *


modules = import_module('LIAFnet.LIAFResNet_34')
config  = modules.Config()
snn34 = LIAFResNet(config)

modules = import_module('LIAFnet.LIAFResNet_18')
config  = modules.Config()
snn18 = LIAFResNet(config)

from LIAFnet.ResNet2D import *
cnn2d18 = resnet18()
cnn2d34 = resnet34()

from LIAFnet.ResNet3D import *

cnn3d18 = resnet18()
cnn3d34 = resnet34()

In [2]:
from LIAF import *
from LIAFnet.LIAFResNet import *
from torch.nn import BatchNorm3d,Conv2d,Linear,Conv3d


time_windows = 8
feature_size = {2:224,64:55,128:28,256:14,512:7,1:224}#chennel:size
feature_size_2 = {64:4,128:2,256:1,512:1}

def cal_LIAFConvCell(m,isliaf=True):
        print(type(m))
        kernel_size = m.kernel.kernel_size[0]
        inchannel =  m.kernel.in_channels
        outchennel = m.kernel.out_channels
        stride = m.kernel.stride[0]
        padding = m.kernel.padding[0]
        n = feature_size[inchannel]
        fp32mut_this = 0
        fp32plus_this = 0
        #Conv
        outsize = math.floor((n + 2 * padding - kernel_size) / stride + 1)
        if isliaf:
            fp32mut_this += inchannel * outchennel * outsize**2*kernel_size**2 
        fp32plus_this += inchannel * outchennel * outsize**2*(kernel_size**2-1)
        #U_update 
        fp32plus_this += outchennel * outsize**2
        #U_spike
        fp32plus_this += outchennel * outsize**2
        fp32mut_this += 2*outchennel * outsize**2
        print('intput:',n,'*',n, 'output:',outsize,'*',outsize)
        return fp32plus_this*time_windows ,fp32mut_this*time_windows

def cal_Temporal_Conv2d(m,isliaf=True):
        print(type(m))
        kernel_size = m.kernel_size[0]
        inchannel =  m.in_channels
        outchennel = m.out_channels
        stride = m.stride[0]
        padding = m.padding[0]
        n = feature_size[inchannel]
        fp32mut_this = 0
        fp32plus_this = 0
        #Conv
        outsize = math.floor((n + 2 * padding - kernel_size) / stride + 1)
        if isliaf:
            fp32mut_this += inchannel * outchennel * outsize**2*kernel_size**2 
        fp32plus_this += inchannel * outchennel * outsize**2*(kernel_size**2-1)
        #U_update 
        fp32plus_this += outchennel * outsize**2
        #U_spike
        fp32plus_this += outchennel * outsize**2
        fp32mut_this += 2*outchennel * outsize**2
        print('intput:',n,'*',n, 'output:',outsize,'*',outsize)
        return fp32plus_this*time_windows, fp32mut_this*time_windows
    
def cal_Conv2d(m):
        print(type(m))
        kernel_size = m.kernel_size[0]
        inchannel =  m.in_channels
        outchennel = m.out_channels
        stride = m.stride[0]
        padding = m.padding[0]
        n = feature_size[inchannel]
        fp32mut_this = 0
        fp32plus_this = 0
        #Conv
        outsize = math.floor((n + 2 * padding - kernel_size) / stride + 1)
        fp32mut_this +=  outchennel *(inchannel *kernel_size**2+1) * outsize**2
        fp32plus_this += inchannel * outchennel * outsize**2*(kernel_size**2-1)
        print('intput:',n,'*',n, 'output:',outsize,'*',outsize,'   ',inchannel,'to',outchennel)
        return fp32plus_this, fp32mut_this
    
def cal_Conv3d(m):
        print(type(m))
        kernel_size = m.kernel_size[1]
        inchannel =  m.in_channels
        outchennel = m.out_channels
        stride = m.stride[1]
        padding = m.padding[1]
        
        depth =  m.kernel_size[0]
        timestride = m.stride[0]
        timespadding = m.stride[0]
        
        n = feature_size[inchannel]
        fp32mut_this = 0
        fp32plus_this = 0
        #Conv
        outtime = feature_size_2[outchennel]
        outsize = math.floor((n + 2 * padding - kernel_size) / stride + 1)
        
        fp32mut_this += inchannel * outchennel * outsize**2*outtime * kernel_size**2*depth
        fp32plus_this += inchannel * outchennel * outsize**2*outtime * (kernel_size**2*depth-1)
        print('intput:',time_windows,'*',n,'*',n, 'output:',outtime,'*',outsize,'*',outsize,'   ',inchannel,'to',outchennel)
        return fp32plus_this, fp32mut_this


def cal_Linear(m):
        print(type(m))
        input_size = m.in_features
        hedden_size =  m.out_features
        #Conv
        fp32mut_this = 0
        fp32plus_this = 0
        fp32mut_this += input_size*input_size * input_size
        fp32plus_this += input_size*input_size * input_size
        print('intput:',input_size,'*',input_size, 'output:',hedden_size,'*',hedden_size)
        return fp32plus_this, fp32mut_this

isliaf = False
def cal_ops(model):
    fp32plus = 0
    fp32mut = 0
    for idx,m in enumerate(model.modules()):
        if isinstance(m,LIAFResNet) or isinstance(m,nn.Sequential):
            continue
        if isinstance(m,LIAFConvCell) :
            fp32plus_this,fp32mut_this = cal_LIAFConvCell(m,isliaf)
            print('fp32+=',fp32plus_this,'fp32*=',fp32mut_this)
        elif isinstance(m,Temporal_Conv2d):
            fp32plus_this,fp32mut_this = cal_Temporal_Conv2d(m,isliaf)
            print('fp32+=',fp32plus_this,'fp32*=',fp32mut_this)
        elif isinstance(m,Conv2d):
            fp32plus_this,fp32mut_this = cal_Conv2d(m)
            print('fp32+=',fp32plus_this,'fp32*=',fp32mut_this)
        elif isinstance(m,Linear):
            fp32plus_this,fp32mut_this = cal_Linear(m)
            print('fp32+=',fp32plus_this,'fp32*=',fp32mut_this)
        elif isinstance(m,Conv3d):
            fp32plus_this,fp32mut_this = cal_Conv3d(m)
            print('fp32+=',fp32plus_this,'fp32*=',fp32mut_this)
        else:
            continue
        fp32plus += fp32plus_this
        fp32mut += fp32mut_this
    return fp32plus , fp32mut


- LIAF18 fp32+= 12668145664 fp32*= 14159104000
- LIAF34 fp32+= 25783130112 fp32*= 28901059584
- 3D-18  fp32+= 12082982912 fp32*= 12492945408
- 3D-34  fp32+= 20671672320 fp32*= 21411969024
- LIF18  fp32+= 12668145664 fp32*= 268636160
- LIF34  fp32+= 25783130112 fp32*= 288479232
- 2D-18  fp32+= 1574797312 fp32*= 1770123264
- 2D-34  fp32+= 3211689984 fp32*= 3611627520

In [3]:

#fp32plus , fp32mut = cal_ops(snn)
#print(type(snn),'fp32+=',fp32plus,'fp32*=',fp32mut)
isliaf = False
fp32plus , fp32mut = cal_ops(cnn2d18)
print(type(cnn2d18),'fp32+=',fp32plus,'fp32*=',fp32mut)

<class 'torch.nn.modules.conv.Conv2d'>
intput: 224 * 224 output: 112 * 112     1 to 64
fp32+= 38535168 fp32*= 40140800
<class 'torch.nn.modules.conv.Conv2d'>
intput: 55 * 55 output: 28 * 28     64 to 64
fp32+= 25690112 fp32*= 28951552
<class 'torch.nn.modules.conv.Conv2d'>
intput: 55 * 55 output: 55 * 55     64 to 64
fp32+= 99123200 fp32*= 111707200
<class 'torch.nn.modules.conv.Conv2d'>
intput: 55 * 55 output: 55 * 55     64 to 64
fp32+= 99123200 fp32*= 111707200
<class 'torch.nn.modules.conv.Conv2d'>
intput: 55 * 55 output: 55 * 55     64 to 64
fp32+= 99123200 fp32*= 111707200
<class 'torch.nn.modules.conv.Conv2d'>
intput: 55 * 55 output: 28 * 28     64 to 128
fp32+= 51380224 fp32*= 57903104
<class 'torch.nn.modules.conv.Conv2d'>
intput: 28 * 28 output: 28 * 28     128 to 128
fp32+= 102760448 fp32*= 115705856
<class 'torch.nn.modules.conv.Conv2d'>
intput: 55 * 55 output: 28 * 28     64 to 128
fp32+= 0 fp32*= 6522880
<class 'torch.nn.modules.conv.Conv2d'>
intput: 28 * 28 output: 28 * 

In [4]:
input_random = torch.zeros([1,2,8,224,224])
cnn3d18(input_random)

torch.Size([1, 64, 4, 55, 55])
torch.Size([1, 64, 4, 55, 55])
torch.Size([1, 64, 4, 55, 55])
torch.Size([1, 64, 4, 55, 55])
torch.Size([1, 128, 2, 28, 28])
torch.Size([1, 128, 2, 28, 28])
torch.Size([1, 128, 2, 28, 28])
torch.Size([1, 128, 2, 28, 28])
torch.Size([1, 256, 1, 14, 14])
torch.Size([1, 256, 1, 14, 14])
torch.Size([1, 256, 1, 14, 14])
torch.Size([1, 256, 1, 14, 14])
torch.Size([1, 512, 1, 7, 7])
torch.Size([1, 512, 1, 7, 7])
torch.Size([1, 512, 1, 7, 7])
torch.Size([1, 512, 1, 7, 7])


tensor([[ 3.8691e-02,  3.9335e-03,  2.6532e-02,  3.3640e-03,  3.9258e-02,
         -4.2711e-02, -2.8663e-02, -6.4670e-03, -2.1399e-02, -3.9675e-02,
         -3.0206e-02,  2.8576e-02, -1.1526e-02, -3.8382e-02, -3.4531e-02,
         -1.4308e-02, -2.7699e-02,  4.2343e-03, -1.3172e-02,  1.9549e-02,
          3.5606e-02, -8.6369e-03,  3.9756e-02,  2.3853e-02, -3.0713e-02,
         -1.9565e-03, -3.5206e-02, -2.4373e-02, -3.3446e-02, -3.1833e-02,
         -1.4571e-02, -2.6690e-02, -7.5055e-03, -9.1792e-03,  8.2165e-03,
          1.7861e-02,  2.3602e-02,  1.1375e-02,  4.2980e-02,  6.1698e-05,
          2.2729e-02,  1.5666e-02,  1.9304e-02, -1.3597e-03, -1.2662e-03,
          3.8828e-02, -3.5047e-04, -8.4883e-03, -3.7326e-02,  5.2753e-03,
          2.1000e-02, -3.9837e-02, -3.3983e-02,  2.8843e-02, -4.4189e-04,
          3.9028e-02,  4.7019e-03,  3.6727e-02, -3.3100e-02, -2.4353e-03,
         -1.5840e-02, -7.5066e-03,  3.5507e-02,  2.5336e-02,  2.6135e-02,
          2.9095e-02, -4.3934e-02, -1.

In [11]:
from torchvision.models import resnet34 as res34
from thop import profile
from thop import clever_format
model = res34()
input = torch.randn(1, 3, 224, 224)
macs, params = profile(model, inputs=(input, ))
macs, params = clever_format([macs, params], "%.3f")
print(macs)

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[WARN] Cannot find rule for <class 'torchvision.models.resnet.BasicBlock'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class 'torchvision.models.resnet.ResNet'>. Treat it as zero Macs and zero Params.
3.671G


In [1]:
from __future__ import print_function
from util.util import lr_scheduler
from datasets.es_imagenet_new import ESImagenet_Dataset
import LIAF
from LIAFnet.LIAFResNet import *
import torch.distributed as dist 
import torch.nn as nn
import argparse, pickle, torch, time, os,sys
from importlib import import_module

##################### Step1. Env Preparation #####################
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##################### Step2. load in dataset #####################
modules = import_module('LIAFnet.LIAFResNet_18')
config  = modules.Config()
config.actFun= LIFactFun.apply
num_epochs = config.num_epochs
batch_size = 20
local_rank = 0
timeWindows = config.timeWindows

epoch = 0
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
training_iter = 0
start_epoch = 0
acc_record = list([])
loss_train_record = list([])
loss_test_record = list([])
train_dataset = ESImagenet_Dataset(mode='train',data_set_path='/data/ES-imagenet-0.18/')
test_dataset = ESImagenet_Dataset(mode='test',data_set_path='/data/ES-imagenet-0.18/')
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=1,pin_memory=True,drop_last=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1,pin_memory=True)

##################### Step3. establish module #####################
snn = LIAFResNet(config)
snn.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(snn.parameters(),
            lr=config.learning_rate)


################step4. training and validation ################


def cal_fire_rate(model):
    fire_rate = 0
    layercount = 0
    fire = 0
    for idx,m in enumerate(model.modules()):
        if isinstance(m,LIAFResNet) or isinstance(m,nn.Sequential):
            continue
        if isinstance(m,LIAFConvCell) or isinstance(m,Temporal_Conv2d):
            fire = m.fire_rate
        else:
            continue
        layercount+= 1
        fire_rate += fire
    return fire_rate/layercount

fire_rate_dict = {}

for epoch in range(num_epochs):
    #training
    fire_rate_list = []
    running_loss = 0
    snn.train()
    start_time = time.time() 
    correct = 0.0
    total = 0.0
    for i, (images, labels) in enumerate(train_loader):
        if ((i+1)<=len(train_dataset)//batch_size):
            snn.zero_grad()
            optimizer.zero_grad()
            outputs = snn(images.type(LIAF.dtype)).cpu()
            labels = labels.view(batch_size)
            loss = criterion(outputs, labels)
            _ , predict = outputs.max(1)
            correct += predict.eq(labels).sum()
            total += float(predict.size(0))
            running_loss += loss.item()
            loss.backward()
            optimizer.step()
            fire_rate = cal_fire_rate(snn)
            print(fire_rate.item())
            fire_rate_list.append(fire_rate)
        training_iter +=1 
    torch.cuda.empty_cache()
    fire_rate_dict[epoch]=fire_rate_list

0.29036131501197815
0.2918887138366699
0.2935800850391388
0.29492294788360596
0.29272714257240295
0.29272374510765076
0.2935844957828522
0.2924903333187103
0.2918911576271057
0.2905900180339813
0.2930019199848175
0.2920590043067932
0.2921254336833954
0.2924834191799164
0.29006755352020264
0.29061999917030334
0.2900073826313019
0.2907888889312744
0.2890413701534271
0.292036771774292
0.28947168588638306
0.2919151186943054
0.29037272930145264
0.29627084732055664
0.2918362617492676
0.2917218506336212
0.2918252944946289
0.2919565439224243
0.2904450297355652
0.290342777967453
0.2904481291770935
0.2896101772785187
0.2894973158836365
0.2911831736564636
0.2882803976535797
0.29161253571510315
0.28969037532806396
0.2891142666339874
0.29127237200737
0.291393905878067
0.29067811369895935
0.28896647691726685
0.2915094494819641
0.2876144051551819
0.2907082140445709
0.2945060431957245
0.2905194163322449
0.2930155098438263
0.2865414321422577
0.2886667847633362
0.29288214445114136
0.2930958569049835
0.2

KeyboardInterrupt: 